<a href="https://colab.research.google.com/github/kk25gb/AIdea-project-file/blob/Siamese-network/Siamese.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import json
import csv
from keras import layers,Input
from keras.models import Model
from urllib.request import urlopen
from sklearn.cluster import KMeans
from keras import regularizers
import pandas as pd
import numpy as np

# Stage 1 資料前處理

## 讀入向量化資料


In [2]:
url = 'https://raw.githubusercontent.com/kk25gb/AIdea-project-file/main/stage1/vectorize_data.json'
response = urlopen(url)
data = json.loads(response.read())
known_assay_labels = list(data.keys())
df = pd.DataFrame(data).T

In [3]:
len(data['1'])

764

## 讀入 True label csv

In [4]:
url = 'https://raw.githubusercontent.com/kk25gb/AIdea-project-file/main/stage1/TrainLabel.csv'
response = urlopen(url)
lines = [l.decode('utf-8') for l in response.readlines()]
true_labels = list(csv.reader(lines))
del true_labels[0]

In [5]:
len(true_labels)

1381

## 建立 False label 資料

In [6]:
import random

input_keys = list(data.keys())
false_labels = []

while len(false_labels) < len(true_labels):
  temp = [random.choice(input_keys),random.choice(input_keys)]
  if temp not in true_labels:
    false_labels.append(temp)

In [7]:
len(false_labels)

1381

## 產生所需資料格式

In [8]:
def data_generator(input,Label=0):
  return [[[data[row[0]],data[row[1]]],Label] for row in input]

In [9]:
# 標籤為True的向量資料對
true_list = data_generator(true_labels,1)

In [10]:
# 標籤為False的向量資料對
false_list = data_generator(false_labels,0)

In [11]:
df_true = pd.DataFrame(true_list)
df_true.head()

,0,1
0,"[[0, 0, 0, 2, 4, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0,...",1
1,"[[0, 0, 0, 2, 4, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0,...",1
2,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0,...",1
3,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...",1
4,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...",1


In [12]:
df_false = pd.DataFrame(false_list)
df_false.head()

,0,1
0,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...",0
1,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...",0
2,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 10, 2...",0
3,"[[0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...",0
4,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 8, 2,...",0


## 混合 Ture / False 標籤的資料

In [13]:
train_list = true_list+false_list

In [14]:
len(train_list)

2762

In [15]:
df_train = pd.DataFrame(train_list)
df_train.head()

,0,1
0,"[[0, 0, 0, 2, 4, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0,...",1
1,"[[0, 0, 0, 2, 4, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0,...",1
2,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0,...",1
3,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...",1
4,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...",1


In [16]:
from sklearn.utils import shuffle
df_train = shuffle(df_train)

In [17]:
df_train.head()

,0,1
1663,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 4, 0,...",0
993,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...",1
2604,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 8, 0,...",0
78,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...",1
2742,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...",0


In [18]:
X = df_train[0]
Y_true = df_train[1]

#### 將輸出標籤改寫為True 與 Fasle 兩個 Column 組成

In [19]:
Y =[[1-true_label,true_label] for true_label in Y_true]

In [20]:
pd.DataFrame(Y).head()

,0,1
0,1,0
1,0,1
2,1,0
3,0,1
4,1,0


### 轉為Numpy陣列

In [21]:
left_x = np.array([row[0] for row in X]).astype('float32')
right_x = np.array([row[1] for row in X]).astype('float32')
y_train = np.array(Y).astype('float32')

In [22]:
pd.DataFrame(left_x).head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,724,725,726,727,728,729,730,731,732,733,734,735,736,737,738,739,740,741,742,743,744,745,746,747,748,749,750,751,752,753,754,755,756,757,758,759,760,761,762,763
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,8.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,2.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,5.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [23]:
pd.DataFrame(right_x).head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,724,725,726,727,728,729,730,731,732,733,734,735,736,737,738,739,740,741,742,743,744,745,746,747,748,749,750,751,752,753,754,755,756,757,758,759,760,761,762,763
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,2.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,3.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [24]:
pd.DataFrame(y_train).head()

,0,1
0,1.0,0.0
1,0.0,1.0
2,1.0,0.0
3,0.0,1.0
4,1.0,0.0


In [25]:
left_x.shape

(2762, 764)

In [26]:
right_x.shape

(2762, 764)

In [27]:
y_train.shape

(2762, 2)

# 建立神經網路模型

參考資料：
* [Image similarity estimation using a Siamese Network with a triplet loss](https://keras.io/examples/vision/siamese_network/)
* [Siamese Network — Unofficial Keras Implementation](https://github.com/gchoi/face-recognition-using-siamese-network)
* [Text-Similarity-Using-Siamese-Deep-Neural-Network](https://github.com/AnjaliDharmik/Text-Similarity-Using-Siamese-Deep-Neural-Network)

## 建構原理
藉由同一個層來一起進行萃取兩篇文章的相似度特徵  
因此需共享權重來訓練神經網路

In [28]:
lstm = layers.LSTM(50,activation="tanh",recurrent_activation="sigmoid",dropout=0.25,recurrent_dropout=0.25,) # 共享權重的神經層

left_input = Input(shape=(764,),name='left_input')
# print(left_input.shape)
left_embedded_text = layers.Embedding(764,100)(left_input) # 輸入向量764；輸入向量764
left_output = lstm(left_embedded_text) 
# print(left_output.shape)

right_input = Input(shape=(764,),name='right_input')
# print(right_input.shape)
right_embedded_text = layers.Embedding(764,100)(right_input)
right_output = lstm(right_embedded_text)
# print(right_output.shape)

merged = layers.concatenate([left_output,right_output],axis=-1) # 將向量串接
# print(merged.shape)
narrow_down_1 = layers.Dense(100,activation='relu', kernel_regularizer=regularizers.l2(0.001))(merged)
narrow_down_1 = layers.Dropout(0.5)(narrow_down_1)
# narrow_down_2 = layers.Dense(382,activation='relu', kernel_regularizer=regularizers.l2(0.001))(narrow_down_1)
# narrow_down_2 = layers.Dropout(0.5)(narrow_down_2)
# narrow_down_3 = layers.Dense(191,activation='relu', kernel_regularizer=regularizers.l2(0.001))(narrow_down_2)
# narrow_down_3 = layers.Dropout(0.5)(narrow_down_3)
# narrow_down_4 = layers.Dense(96,activation='relu', kernel_regularizer=regularizers.l2(0.001))(narrow_down_3)
# narrow_down_4 = layers.Dropout(0.5)(narrow_down_4)
predictions = layers.Dense(2,activation='sigmoid')(narrow_down_1)
model = Model([left_input,right_input],predictions)

In [29]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 left_input (InputLayer)        [(None, 764)]        0           []                               
                                                                                                  
 right_input (InputLayer)       [(None, 764)]        0           []                               
                                                                                                  
 embedding (Embedding)          (None, 764, 100)     76400       ['left_input[0][0]']             
                                                                                                  
 embedding_1 (Embedding)        (None, 764, 100)     76400       ['right_input[0][0]']            
                                                                                              

In [30]:
model = Model([left_input,right_input],predictions)
model.compile(optimizer='rmsprop',loss='binary_crossentropy',metrics=['acc'])

In [31]:
# 訓練方法2；使用dict將送入資料進行訓練，鍵為Input層的名稱，值為Numpy的值
model.fit({'left_input':left_x,'right_input':right_x},y_train,epochs=10,batch_size=128,validation_split=0.25)

Epoch 1/10
17/17 [==============================] - 155s 9s/step - loss: 0.7742 - acc: 0.4983 - val_loss: 0.7588 - val_acc: 0.4993
Epoch 2/10
17/17 [==============================] - 149s 9s/step - loss: 0.7506 - acc: 0.5060 - val_loss: 0.7411 - val_acc: 0.4993
Epoch 3/10
17/17 [==============================] - 149s 9s/step - loss: 0.7353 - acc: 0.4983 - val_loss: 0.7274 - val_acc: 0.5007
Epoch 4/10
17/17 [==============================] - 147s 9s/step - loss: 0.7227 - acc: 0.4998 - val_loss: 0.7166 - val_acc: 0.4993
Epoch 5/10
17/17 [==============================] - 145s 9s/step - loss: 0.7130 - acc: 0.4969 - val_loss: 0.7084 - val_acc: 0.4993
Epoch 6/10
17/17 [==============================] - 148s 9s/step - loss: 0.7056 - acc: 0.4940 - val_loss: 0.7024 - val_acc: 0.4993
Epoch 7/10
17/17 [==============================] - 148s 9s/step - loss: 0.7005 - acc: 0.4978 - val_loss: 0.6984 - val_acc: 0.4993
Epoch 8/10
17/17 [==============================] - 147s 9s/step - loss: 0.6978 - a

# 載入Stage 2 比對資料


In [32]:
url = 'https://raw.githubusercontent.com/kk25gb/AIdea-project-file/main/stage2/vectorize_data.json'
response = urlopen(url)
predict_data = json.loads(response.read())
predict_assay_label = list(predict_data.keys())
df = pd.DataFrame(predict_data).T

In [33]:
known_assay = np.array(list(data.values())).astype('float32')
compared_assay = np.array(list(predict_data.values())).astype('float32')

In [34]:
len(known_assay)

559

In [36]:
# predict_assay_label

['1001',
 '1004',
 '1006',
 '1008',
 '1009',
 '1012',
 '1014',
 '1019',
 '102',
 '1022',
 '1024',
 '1027',
 '1028',
 '1029',
 '1032',
 '1034',
 '1036',
 '1038',
 '1040',
 '1048',
 '1050',
 '1051',
 '1053',
 '1060',
 '1061',
 '1062',
 '1063',
 '1064',
 '1070',
 '1072',
 '1080',
 '1081',
 '1084',
 '1085',
 '1089',
 '109',
 '1091',
 '1092',
 '1094',
 '1095',
 '1097',
 '1098',
 '1105',
 '1106',
 '111',
 '1113',
 '112',
 '1121',
 '1123',
 '1124',
 '1129',
 '1131',
 '1135',
 '1137',
 '114',
 '1140',
 '1143',
 '1156',
 '1158',
 '116',
 '1162',
 '1163',
 '1164',
 '1175',
 '1177',
 '1182',
 '1188',
 '1196',
 '1197',
 '1201',
 '1202',
 '1203',
 '1204',
 '1205',
 '1209',
 '1213',
 '1216',
 '1219',
 '122',
 '1225',
 '1227',
 '123',
 '1231',
 '1233',
 '1238',
 '124',
 '1245',
 '125',
 '1251',
 '1252',
 '1254',
 '1268',
 '1269',
 '1273',
 '1277',
 '128',
 '1284',
 '1285',
 '1287',
 '1288',
 '1295',
 '1296',
 '1299',
 '13',
 '130',
 '1303',
 '1306',
 '1307',
 '1313',
 '1316',
 '1317',
 '1318',
 '132'

### 錯誤解決
* [WARNING: WARNING:tensorflow:Model was constructed with shape (None, 150) , but it was called on an input with incompatible shape (None, 1)](https://stackoverflow.com/questions/61656444/warning-warningtensorflowmodel-was-constructed-with-shape-none-150-but-i)

In [ ]:
len(compared_assay[0])

In [ ]:
# list(compared_assay[0])

In [ ]:
predict_assay = [list(compared_assay[0])]*len(known_assay)
len(predict_assay)

In [ ]:
pd.DataFrame(predict_assay).head()

In [ ]:
def judgement(input):
  print(len(input))
  results = []
  for i in range(len(input)):
    if input[i][1]>input[i][0]:
      results.append(known_assay_labels[i])
  return results

In [ ]:
results = {}
for label ,predict_row in zip(predict_assay_label,compared_assay):
  predict_assay = np.array([list(predict_row)]*len(compared_assay))
  # print(len(predict_assay))
  # print(len(known_assay))
  results[label] = model.predict({'left_input':predict_assay,'right_input':compared_assay})
  # break
  

In [ ]:
pd.DataFrame(results[predict_assay_label[0]]).head()

In [ ]:
final_dict = {}
for label ,result in results.items():
   final_dict[label] = judgement(result)

In [ ]:
results[predict_assay_label[0]][0][0]

In [ ]:
judgement(results[predict_assay_label[0]])

In [ ]:
final_dict